In [1]:
!pip install langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.5 MB/s eta 0:00:00


In [2]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

# Using Low Temp for consistent comparison
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

Enter your Google API Key: ··········


In [3]:
# The Task: Reject a candidate for a job.
task = "Write a rejection email to a candidate."

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)

--- LAZY PROMPT ---
Here are a few options for a rejection email, ranging from a standard template to one for a candidate who interviewed. Choose the one that best fits your situation.

---

**Option 1: Standard Rejection (No Interview)**

This is suitable for candidates who applied but were not selected for an interview.

**Subject: Update on Your Application for [Job Title] at [Company Name]**

Dear [Candidate Name],

Thank you for your interest in the [Job Title] position at [Company Name] and for taking the time to submit your application.

We received a large number of highly qualified applications for this role. While your qualifications are impressive, we have decided to move forward with other candidates whose profiles were a closer match for the specific requirements of this position at this time.

We appreciate you considering [Company Name] as a potential employer and wish you the best of luck in your job search and future endeavors.

Sincerely,

[Your Name]
[Your Title]
[Co

In [4]:
structured_prompt = """
# Context
You are an HR Manager at a quirky startup called 'RocketBoots'.

# Objective
Write a rejection email to a candidate named Bob.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'we found someone better'. Say 'the role changed'.
3. Sign off with 'Keep flying'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
Hi Bob,

Thank you for your interest in RocketBoots.

We appreciate you taking the time to interview. However, the role's requirements have changed significantly since your application. We wish you the best in your job search.

Keep flying.


In [5]:
structured_prompt = """
# Context
You are a Senior Python Developer who writes clean, efficient, and production-ready code.

# Objective
Write a Python function that reverses a given string.

# Constraints
1. The function must use recursion.
2. Do NOT use slicing (e.g., [::-1]).
3. Do NOT use built-in reverse functions.

# Style Requirements
1. Include detailed docstrings following PEP 257 conventions.
2. The docstring must include:
   - A clear description of the function.
   - Parameters section.
   - Returns section.
   - Example usage.
3. Use type hints.
4. Use meaningful variable names.

# Output Format
Return only the Python function inside a code block.
No additional explanations outside the code.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)


--- STRUCTURED PROMPT ---
```python
def reverse_string_recursive(input_string: str) -> str:
    """
    Reverses a given string using a recursive approach.

    This function takes a string and returns a new string with its characters
    in reverse order. It achieves this by recursively processing the string,
    taking the first character and appending it to the end of the
    recursively reversed remainder of the string.

    Args:
        input_string: The string to be reversed.

    Returns:
        The reversed string.

    Examples:
        >>> reverse_string_recursive("hello")
        'olleh'
        >>> reverse_string_recursive("Python")
        'nohtyP'
        >>> reverse_string_recursive("")
        ''
        >>> reverse_string_recursive("a")
        'a'
    """
    # Base case: If the string is empty or contains only one character,
    # it is already reversed.
    if len(input_string) <= 1:
        return input_string

    # Recursive step:
    # 1. Extract the first cha

In [6]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

In [7]:
prompt_zero = "Combine 'Angry' and 'Hungry' into a funny new word."
print(f"Zero-Shot: {llm.invoke(prompt_zero).content}")

Zero-Shot: The most common and widely accepted word for this is **Hangry**!

It's already a very popular and funny term for being irritable due to hunger.

If you're looking for something *different* than "Hangry," how about:

*   **Grangry** (combining "grumpy" or "growling" with angry and hungry)
*   **Grungry** (a bit more "grungy" or disheveled in their hungry anger)
*   **Cangry** (from "cranky" + angry + hungry)


In [8]:
prompt_few = """
Combine words into a funny new word. Give a sarcastic definition.

Input: Breakfast + Lunch
Output: Brunch (An excuse to drink alcohol before noon)

Input: Chill + Relax
Output: Chillax (What annoying people say when you are panic attacks)

Input: Angry + Hungry
Output:
"""
print(f"Few-Shot: {llm.invoke(prompt_few).content}")

Few-Shot: Output: Hangry (A convenient medical diagnosis that excuses all your terrible behavior until someone brings you a snack.)


In [9]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [10]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 1. Our Database of Examples
examples = [
    {"input": "The internet is down.", "output": "We are observing connectivity latency."},
    {"input": "This code implies a bug.", "output": "The logic suggests unintended behavior."},
    {"input": "I hate this feature.", "output": "This feature does not align with my preferences."},
]

# 2. Template for ONE example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 3. The Few-Shot Container
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

# 4. The Final Chain
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator. Rewrite the input to sound professional."),
    few_shot_prompt,      # Inject examples here
    ("human", "{text}")
])

chain = final_prompt | llm

print(chain.invoke({"text": "This app sucks."}).content)

This application is currently exhibiting suboptimal performance.
